# Enums and Pattern Matching
---

In this chapter, we’ll look at enumerations, also referred to as enums. Enums  
allow you to define a type by enumerating its possible variants. First we’ll  
define and use an enum to show how an enum can encode meaning along with data.  
Next, we’ll explore a particularly useful enum, called Option, which expresses  
that a value can be either something or nothing. Then we’ll look at how pattern  
matching in the match expression makes it easy to run different code for  
different values of an enum. Finally, we’ll cover how the if let construct is  
another convenient and concise idiom available to handle enums in your code.

Defining an Enum
---

Where structs give you a way of grouping together related fields and data, like  
a Rectangle with its width and height, enums give you a way of saying a value is  
one of a possible set of values. For example, we may want to say that Rectangle  
is one of a set of possible shapes that also includes Circle and Triangle. To do  
this, Rust allows us to encode these possibilities as an enum.

#### -- Enum Examples

Let’s look at a situation we might want to express in code and see why enums are  
useful and more appropriate than structs in this case. Say we need to work with  
IP addresses. Currently, two major standards are used for IP addresses: version  
four and version six. Because these are the only possibilities for an IP address  
that our program will come across, we can enumerate all possible variants, which  
is where enumeration gets its name.

Any IP address can be either a version four or a version six address, but not  
both at the same time. That property of IP addresses makes the enum data  
structure appropriate because an enum value can only be one of its variants.  
Both version four and version six addresses are still fundamentally IP  
addresses, so they should be treated as the same type when the code is handling  
situations that apply to any kind of IP address.

We can express this concept in code by defining an `IpAddrKind` enumeration and  
listing the possible kinds an IP address can be, V4 and V6. These are the  
variants of the enum:

```rust
enum IpAddrKind{
    V4,
    V6
}
```

> `IpAddrKind` is now a custom data type that we can use elsewhere in our code.



### Enum Values

We can create instances of each of the two variants of IpAddrKind like this:

```rust
    let four = IpAddrKind::V4;
    let six = IpAddrKind::V6;
```

> Note that the variants of the enum are namespaced under its identifier, and we  
use a double colon to separate the two. This is useful because now both values  
`IpAddrKind::V4` and `IpAddrKind::V6` are of the same type: `IpAddrKind`. We  
can then, for instance, define a function that takes any `IpAddrKind`:  

```rust
// function that takes any IpAddrKind enum
fn route(ip_kind:IpAddrKind) {}
```

And we can call this function with either variant:

```rust
    route(IpAddrKind::V4)
    route(IpAddrKind::V6)
```

Using enums has even more advantages. Thinking more about our IP address type,  
at the moment we don’t have a way to store the actual IP address data; we only  
know what kind it is. Given that you just learned about structs in Chapter 5,  
you might be tempted to tackle this problem with structs as shown in Listing 6-1.

In [2]:
enum IpAddrKind { // enum will be stored on the stack
    V4,
    V6
}

// Struct wraps enum
struct IpAddr {
    // @note : enum inside a struct
    kind: IpAddrKind,   // enum is stack allocated
    address: String     // String is heap allocated
}
    // scope is pushed onto the stack frame
{   // entering scope
    let home = IpAddr {                     // binding of 'home; created, 
                                            // assigned instance of IpAddr
        kind: IpAddrKind::V4,               // kind is IpAddrKind::V4 added to stack
        address: String::from("127.0.0.1")  // address assigned new string on heap
    };

    let loopback = IpAddr {                 // binding of 'loopback' created
                                            // assigned instance of IpAddr
        kind: IpAddrKind::V6,               // kind is IpAddrKind::V6 added to stack
        address: String::from("::1")        // address assigned new string on heap
    };
}   // existing scope
    // loopback are out of scope 
    // - kind and address are dropped from the stack
    // - String::from("::1") is dropped from the heap
    // home is out of scope
    // - kind and address are dropped from the stack
    // - String::from("127.0.0.1") is dropped from the heap
    // scope is popped off the stack frame

()

> Listing 6-1: Storing the data and IpAddrKind variant of an IP address using a  
struct

#### -- Enum Inside Struct Pattern

Here, we’ve defined a struct IpAddr that has two fields:  

- a kind field that is of type IpAddrKind (the enum we defined previously) and  
- an address field of type String.  

We have two instances of this struct.  
- The first is home, and it has the value IpAddrKind::V4 as its kind with  
associated address data of 127.0.0.1.  
- The second instance is loopback. It has the other variant of IpAddrKind as  
its kind value, V6, and has address ::1 associated with it.  

We’ve used a struct to bundle the kind and address values together, so now the  
variant is associated with the value.

#### -- Enum Without Struct Pattern

However, representing the same concept using just an enum is more concise:  
rather than an enum inside a struct, we can **put data directly into each enum  
variant**. This new definition of the IpAddr enum says that both V4 and V6  
variants will have associated String values:

In [10]:
enum IpAddr {
    V4(String),
    V6(String)
}

{
    let home = IpAddr::V4(String::from("127.0.0.1"));
    let loopback = IpAddr::V6(String::from("::1"));
}

()

#### -- Enum Embedded

We attach data to each variant of the enum directly, so there is no need for an  
extra struct. Here, it’s also easier to see another detail of how enums work:  
the name of each enum variant that we define also becomes a function that  
constructs an instance of the enum. That is, IpAddr::V4() is a function call  
that takes a String argument and returns an instance of the IpAddr type. We  
automatically get this constructor function defined as a result of defining the  
enum.

There’s another advantage to using an enum rather than a struct: each variant  
can have different types and amounts of associated data. Version four IP  
addresses will always have four numeric components that will have values between  
0 and 255. If we wanted to store V4 addresses as four u8 values but still  
express V6 addresses as one String value, we wouldn’t be able to with a struct.  
Enums handle this case with ease:

In [13]:
// Another advanatge to enum vs struct
enum IpAddr{
    // Each variant can have DIFFERENT types and AMOUNTS of associated data!
    V4(u8, u8, u8, u8),     // 4 u8 values
    V6(String)              // 1 String value
}

{
    // Each enum variant passed different arguments
    let home = IpAddr::V4(127, 0, 0, 1);            // 4 u8 values
    let loopback = IpAddr::V6(String::from("::1")); // 1 String value
}

()

We’ve shown several different ways to define data structures to store version  
four and version six IP addresses. However, as it turns out, wanting to store IP  
addresses and encode which kind they are is so common that the standard library  
has a definition we can use! Let’s look at how the standard library defines  
IpAddr: it has the exact enum and variants that we’ve defined and used, but it  
embeds the address data inside the variants in the form of two different  
structs, which are defined differently for each variant:

```rust
struct Ipv4Addr {
    // --snip--
}

struct Ipv6Addr {
    // --snip--
}

// Even data like custom struct can be embedded in enum (including other enums!)
// not just built-ins
enum IpAddr{
    V4(Ipv4Addr),
    V6(Ipv6Addr)
}
```

#### -- Enum Variant entry can hold ANY kind of Data Type

This code illustrates that you can put any kind of data inside an enum variant:

- strings, 
- numeric types, or 
- structs, for example. 
- You can even include another enum! 

Also, standard library types are often not much more complicated than what you  
might come up with.

Note that even though the standard library contains a definition for IpAddr, we  
can still create and use our own definition without conflict because we haven’t  
brought the standard library’s definition into our scope. We’ll talk more about  
bringing types into scope in Chapter 7.

#### -- Enum with Completely Variant Types

Let’s look at another example of an enum in Listing 6-2: this one has a wide  
variety of types embedded in its variants.

```rust
// enum with 4 completely differenc types!
enum Message {
    Quit,                       // no data
    Move { x: i32, y: i32 },    // struct with named fields
    Write(String),              // func with 1 String arg
    ChangeColor(i32, i32, i32)  // func with 3 i32 args
}
```

> Listing 6-2: A Message enum whose variants each store different amounts and  
types of values

This enum has four variants with different types:

- Quit has no data associated with it at all.
- Move has named fields, like a struct does.
- Write includes a single String.
- ChangeColor includes three i32 values.

#### -- Variant Types Unwrapped as Struct

Defining an enum with variants such as the ones in Listing 6-2 is similar to  
defining different kinds of struct definitions, except the enum doesn’t use the  
struct keyword and all the variants are grouped together under the Message type.  
The following structs could hold the same data that the preceding enum variants  
hold:

```rust
struct QuitMessage;                         // unit struct
struct MoveMessage {
    x: i32,
    y: i32,
}
struct WriteMessage(String);                // tuple struct
struct ChangeColorMessage(i32, i32, i32);   // tuple struct

// But we sould need different function overloads to handle  
// EACH SEPARATE struct vs Handling ONE SINGLE enum variants
```

But if we used the different structs, each of which has its own type, we  
couldn’t as easily define a function to take any of these kinds of messages as  
we could with the Message enum defined in Listing 6-2, which is a single type.

#### -- Enum Supports Method Calls

There is one more similarity between enums and structs: just as we’re able to  
define methods on structs using impl, we’re also able to define methods on  
enums. Here’s a method named call that we could define on our Message enum:

```rust
impl Message{
    fn call(&self){
        // method body defined here
    }
}

let m = Message::Write(String::from("hello"));
m.call();
```

The body of the method would use self to get the value that we called the method  
on. In this example, we’ve created a variable `m` that has the value  
`Message::Write(String::from("hello"))`, and that is what self will be in the body  
of the call method when `m.call()` runs.

In [15]:
#[derive(Debug)]
enum Message{
    Quit,
    Write(String),
    Move{x: i32, y: i32},
    ChangeColor(i32, i32, i32)
}

impl Message{
    fn call(&self){
        println!("Message::call {:?}", self);
    }
}

{
    let m = Message::Write(String::from("hello ENUMS"));
    m.call();

    let q = Message::Quit;
    q.call();

    let mv = Message::Move{x: 1, y: 2};
    mv.call();

    let color = Message::ChangeColor(0, 1, 0);
    color.call();
}

Message::call Write("hello ENUMS")
Message::call Quit
Message::call Move { x: 1, y: 2 }
Message::call ChangeColor(0, 1, 0)


()

### The Option Enum and Its Advantages Over Null Values

Let’s look at another enum in the standard library that is very common and  
useful: `Option`.

This section explores a case study of `Option`, which is another enum defined by  
the standard library. The `Option` type encodes the very common scenario in which  
a value could be `something` or it could be `nothing`.

For example, if you request the first item in a non-empty list, you would get a 
value. If you request the first item in an empty list, you would get nothing.  
Expressing this concept in terms of the type system means the compiler can check  
whether you’ve handled all the cases you should be handling; this functionality  
can prevent bugs that are extremely common in other programming languages.

Programming language design is often thought of in terms of which features you  
include, but the features you exclude are important too. Rust doesn’t have the  
null feature that many other languages have. Null is a value that means there is  
no value there. In languages with null, variables can always be in one of two  
states: null or not-null.

#### -- Null Reference : The Billion Dollar Mistake

In his 2009 presentation “Null References: The Billion Dollar Mistake,” Tony  
Hoare, the inventor of null, has this to say:

```css
I call it my billion-dollar mistake. At that time, I was designing the first  
comprehensive type system for references in an object-oriented language. My  
goal was to ensure that all use of references should be absolutely safe, with  
checking performed automatically by the compiler. But I couldn’t resist the  
temptation to put in a null reference, simply because it was so easy to  
implement. This has led to innumerable errors, vulnerabilities, and system  
crashes, which have probably caused a billion dollars of pain and damage in the  
last forty years.
```

The problem with null values is that if you try to use a null value as a  
not-null value, you’ll get an error of some kind. Because this null or not-null  
property is pervasive, it’s extremely easy to make this kind of error.

However, the concept that null is trying to express is still a useful one: a  
null is a value that is currently invalid or absent for some reason.

#### -- Option<T> instead of Null

The problem isn’t really with the concept but with the particular implementation.  
As such, `Rust does not have nulls`, but it does have an enum that can encode the  
concept of a value being present or absent. This enum is `Option<T>`, and it is  
defined by the standard library as follows:

```rust
enum Option<T> {
    None,
    Some<T>,
}
```

The `Option<T>` enum is so useful that it’s even included in the prelude; you  
don’t need to bring it into scope explicitly. Its variants are also included in  
the prelude: you can use `Some` and `None` directly without the Option:: prefix.  
The `Option<T>` enum is still just a regular enum, and `Some(T)` and `None` are  
still variants of type `Option<T>`.

#### -- Some<T> and None

The <T> syntax is a feature of Rust we haven’t talked about yet. It’s a generic  
type parameter, and we’ll cover generics in more detail in Chapter 10. For now,  
all you need to know is that <T> means that the `Some` variant of the `Option`  
enum can hold one piece of data of any type, and that each concrete type that  
gets used in place of T makes the overall `Option<T>` type a different type.  
Here are some examples of using Option values to hold number types and string  
types:

```rust
    let some_number = Some(5);
    let some_char = Some('e');

    let absent_number: Option<i32> = None;
```

The type of some_number is Option<i32>. The type of some_char is Option<char>,  
which is a different type. Rust can infer these types because we’ve specified a  
value inside the Some variant. For absent_number, Rust requires us to annotate  
the overall Option type: the compiler can’t infer the type that the  
corresponding Some variant will hold by looking only at a None value. Here, we  
tell Rust that we mean for absent_number to be of type Option<i32>.